I need a small testing playground that won't crash on my laptop, so I'm creating one here for the 10000 frame centriole data.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
filename = Path('../test-data/acTub_COT_gain100_30ms/acTub_COT_gain100_30ms.csv')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

In [202]:
df.describe()

,x,x-uncertainty,y,y-uncertainty,frame,ad-count,chi-square,background
count,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000,214717.000000
mean,2425.196663,35.420145,2182.946881,35.420145,5172.075346,6288.638717,1029.127987,744.422318
std,1595.345619,20.475942,1322.208205,20.475942,2777.221724,15586.487514,2896.254531,312.086668
min,0.000001,0.847267,0.000002,0.847267,0.000000,338.324000,50.035500,523.371000
25%,965.218000,20.430100,982.750000,20.430100,2904.000000,577.636000,222.657000,613.929000
50%,2403.570000,38.192700,2206.350000,38.192700,5165.000000,772.204000,304.708000,656.426000
75%,3892.590000,50.197000,3461.780000,50.197000,7563.000000,1596.560000,433.924000,730.572000
max,4900.000000,80.431400,4200.000000,80.431400,9999.000000,177421.000000,58298.300000,7147.530000


In [8]:
import importlib

In [68]:
maxSize = np.max([df['x'].max(),df['y'].max()])
pixelSize = 100 # nm

fig = plt.figure()
ax  = fig.add_subplot(111)
ax.hist2d(df['x'], df['y'], bins = maxSize / 100)
ax.invert_yaxis()
plt.show()

In [188]:
plt.close()

In [263]:
importlib.reload(ds)

<module 'DataSTORM.processors' from '/home/kmdouglass/src/DataSTORM/DataSTORM/processors.py'>

In [264]:
corr = ds.FiducialDriftCorrect(mergeRadius      = 90,
                               offTime          = 3,
                               minSegmentLength = 30,
                               searchRegions    = [{'xMin' : 1500,
                                                    'xMax' : 1700,
                                                    'yMin' : 2500,
                                                    'yMax' : 2700}])

In [265]:
corr.iSearch(df)

Space bar pressed!
Closed Figure!


/home/kmdouglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/matplotlib/backend_bases.py:2435: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [266]:
plt.close()

In [ ]:
df2 = corr(df)

Frame 2752: 1 trajectories present


In [256]:
df2.describe()

,x,x-uncertainty,y,y-uncertainty,frame,ad-count,chi-square,background,dx,dy
count,223436.000000,223436.000000,223436.000000,223436.000000,223436.000000,223436.000000,223436.000000,223436.000000,223436.000000,223436.000000
mean,2379.285413,34.194471,2220.994594,34.194471,5171.169713,6584.816046,1055.514960,746.688561,12.807199,-23.691409
std,1572.419678,20.974551,1298.629047,20.974551,2779.429342,15434.186445,2892.619185,308.147069,6.774924,21.231652
min,-23.002722,0.847267,-4.215303,0.847267,0.000000,338.324000,50.035500,523.371000,-8.198933,-62.089510
25%,1018.574789,14.514525,1047.561053,14.514525,2900.000000,584.839000,225.625500,615.778750,12.484288,-44.480143
50%,2252.450154,37.285350,2344.172196,37.285350,5164.000000,792.340000,311.724000,659.936000,14.939771,-22.717404
75%,3807.484024,49.638325,3447.863262,49.638325,7566.000000,2387.012500,506.621500,737.033750,16.560039,-1.959924
max,4908.198702,80.431400,4262.089510,80.431400,9999.000000,177421.000000,58298.300000,7147.530000,23.009661,4.216820


In [228]:
corr.searchRegions

[{'xMax': 1630.0403218824745,
  'xMin': 1469.5060474140912,
  'yMax': 2668.7500006947203,
  'yMin': 2450.0000007939661}]

In [186]:
corr.resetSearchRegions()
corr.searchRegions

[{'xMax': None, 'xMin': None, 'yMax': None, 'yMin': None}]

In [262]:
pd.concat([df, corr.fiducialTrajectories[0]])

,x,x-uncertainty,y,y-uncertainty,frame,ad-count,chi-square,background
0,2965.690000,1.19064,1611.600,1.19064,0,115853.000,26381.200,4540.720
1,1596.610000,2.11862,2390.740,2.11862,0,37762.300,5107.840,1643.580
2,4442.210000,3.25058,1329.220,3.25058,0,22300.000,2275.080,2184.290
3,3743.940000,2.59380,238.015,2.59380,0,26891.500,1241.820,1423.240
4,1173.800000,2.57289,434.643,2.57289,0,25410.600,1026.570,1079.120
5,0.154892,4.24897,3091.840,4.24897,0,12885.500,752.045,1230.390
6,4776.000000,6.66218,4199.970,6.66218,0,8117.410,279.646,1627.930
7,3735.760000,6.93152,4149.000,6.93152,0,7166.770,543.763,1343.180
8,4639.690000,27.59580,3655.940,27.59580,0,1655.810,1206.500,1559.450
9,565.407000,11.56060,1940.960,11.56060,0,3561.010,595.158,1053.050


223393